In [31]:
def importlist(inputtext):
    import numpy as np
    list_of_lists = []

    with open(inputtext) as f:
        for line in f:
            inner_list = line.split("\n")
            doubleInner = [liner.strip() for liner in inner_list[0].split(" ")]
            if doubleInner[-1] == '':
                doubleInner.pop(-1)
            list_of_lists.append(doubleInner)
    flag = False
    holder = []
    objectives = []
    attributes = []
    for part in list_of_lists:
        part = list(map(int,part))
        if part[0] < 0:
            # it is an obj value
            objectives.append(part)
            flag = True
            continue
        if flag:
            # it is constraints
            attributes.append(part)
            continue
        #it is the first guys
        holder.append(part[0])
    itemCount = holder[0]
    b = holder[1:]
    return itemCount, np.array(b), np.array(objectives).reshape(-1,itemCount), np.array(attributes).transpose()

In [36]:
n,b,c,a = importlist("2D2KCompinput.txt")
import time

In [43]:
def rectangular(c,a,b):
    import gurobipy as gp 
    from gurobipy import GRB
    import numpy as np
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))
    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    if m.status != 2:
        return [],1
    lexConstraint = round(m.objVal)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = round(m.objVal)
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = round(m.objVal)
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = round(m.objVal)
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    if NW == SE:
        return found, numRect
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        print(len(rectangles),"len")
        #6&7
        R = rectangles.pop(0)
        if R[1][0] - R[0][0] == 1:
            continue
        if R[0][1] - R[1][1] == 1:
            continue
        print(R,"R")
        #8
        R2 = [(R[0][0] , (R[0][1]+R[1][1])/2.0), R[1] ] 
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = round(m.objVal)
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = round(m.objVal)
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            print(tempNW,"tempNW")
            print(R2,"R2")
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        #13  
        R3 = [R[0],      (tempNW[0] - 1, ((R[0][1] + R[1][1])/2.0) + .001)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R3[1][1], name = 'bottom bound')
        
        m.optimize()
        
        z1=round(m.objVal)
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = round(m.objVal)
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            print(tempSE,"tempSE")
            print(R3,"R3")
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 2
        if len(found) % 10 == 0:
            print(len(found))
        if len(found) > 1100:
            print(len(found),len(rectangles))
    return found, numRect

######################################################################################################################


def rectangularThirds(c,a,b):
    import gurobipy as gp 
    from gurobipy import GRB
    import numpy as np
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))

    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    if m.status != 2:
        return [],1
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    if NW == SE:
        return found, numRect
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
        #if R[0][0] >= R[1][0] or R[1][1] >= R[0][1]:
         #   continue
     
        #8
        delta = (R[0][1] - R[1][1])/3
        R2 = [(R[0][0] , (R[0][1]-(2*delta)))    , R[1]]
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        #m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R2[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) >= R2[0][0], name = 'left bound')
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[1][1], name = 'bottom bound')
        
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = round(m.objVal)
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = round(m.objVal)
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #old = m.getConstrByName('left bound')
        #m.remove(old)
        #old = m.getConstrByName('right bound')
        #m.remove(old)
        old = m.getConstrByName('bottom bound')
        m.remove(old)
        #9
        tempNW = (z1,hold)
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        if tempNW[0] == R2[0][0]:
            continue
        
        #EXCLUDE BOTTOM, INCLUDE TOP
        
        
        R3 =[    (R[0][0] , R[0][1]-delta-.001)    , (tempNW[0]-1,R2[0][1] + .001)]
        
        #NW 
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R3[1][1], name = 'bottom bound')
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R3[0][1], name = 'top bound')
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) >= R3[0][0], name = 'left bound')
        m.optimize()
        if m.status == 2:
            lexConstraint = round(m.objVal)
            m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
            m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
            m.optimize()
            hold = round(m.objVal)
            NW = (lexConstraint,hold)
    
        #SE 
            old = m.getConstrByName('temp')
            m.remove(old)
            m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
            m.optimize()
            lexConstraint = round(m.objVal)
            m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
            m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
            m.optimize()
            hold = round(m.objVal)
            SE = (hold, lexConstraint)
            old = m.getConstrByName('temp')
            m.remove(old)
            old = m.getConstrByName('bottom bound')
            m.remove(old)
            old = m.getConstrByName('top bound')
            m.remove(old)
            old = m.getConstrByName('right bound')
            m.remove(old)
            old = m.getConstrByName('left bound')
            m.remove(old)
            
            found.append(list(SE))
            if NW != SE:
                found.append(list(NW))
                rectangles.append([NW,SE])
            if NW[0] == R2[0][0]:
                continue
                
        else:
            NW = tempNW
            old = m.getConstrByName('bottom bound')
            m.remove(old)
            old = m.getConstrByName('top bound')
            m.remove(old)
            old = m.getConstrByName('right bound')
            m.remove(old)
            old = m.getConstrByName('left bound')
            m.remove(old)
        
        
        #13
        #KEEP tempNW[0] IN MIND TO BE CHANGED BY R3 (MIDDLE)
        R4 = [R[0],      (NW[0] - 1, R[0][1]-delta)]
        
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R4[1][1], name = 'bottom bound')
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R4[0][1], name = 'top bound')
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R4[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) >= R4[0][0], name = 'left bound')
        
        m.optimize()
        z1=round(m.objVal)
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = round(m.objVal)
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('bottom bound')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('left bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 3
    return found, numRect

##########################################################################################################################


def rectangularRand(c,a,b):
    import gurobipy as gp 
    from gurobipy import GRB
    import numpy as np
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))
    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    if m.status != 2:
        return [],1
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    if NW == SE:
        return found, numRect
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
        
        cut = np.random.uniform(.2,.8)
        mid = cut*(R[0][1])+(1-cut)*(R[1][1])
     
        #8
        R2 = [(R[0][0] , mid)    , R[1] ] 
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        #13 
        R3 = [R[0],    (tempNW[0] - 1, mid)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[0][1] , name = 'bottom bound')
        
        m.optimize()
        
        z1=m.objVal
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 2
        if len(found) % 10 == 0:
            print(len(found))
        if len(found) > 1100:
            print(len(found),len(rectangles))
    return found, numRect

In [44]:
found, _ = rectangular(c,a,b)

1 len
[(-15175, -10832), (-11310, -14354)] R
(-14620, -12594) tempNW
[(-15175, -12593.0), (-11310, -14354)] R2
(-14621, -12589) tempSE
[(-15175, -10832), (-14621, -12592.999)] R3
2 len
[(-14620, -12594), (-11310, -14354)] R
(-13843, -13476) tempNW
[(-14620, -13474.0), (-11310, -14354)] R2
(-13844, -13473) tempSE
[(-14620, -12594), (-13844, -13473.999)] R3
3 len
[(-15175, -10832), (-14621, -12589)] R
(-15032, -11712) tempNW
[(-15175, -11710.5), (-14621, -12589)] R2
(-15033, -11707) tempSE
[(-15175, -10832), (-15033, -11710.499)] R3
4 len
[(-13843, -13476), (-11310, -14354)] R
(-13213, -13915) tempNW
[(-13843, -13915.0), (-11310, -14354)] R2
(-13216, -13913) tempSE
[(-13843, -13476), (-13214, -13914.999)] R3
10
5 len
[(-14620, -12594), (-13844, -13473)] R
(-14281, -13034) tempNW
[(-14620, -13033.5), (-13844, -13473)] R2
(-14282, -13031) tempSE
[(-14620, -12594), (-14282, -13033.499)] R3
6 len
[(-15032, -11712), (-14621, -12589)] R
(-14859, -12151) tempNW
[(-15032, -12150.5), (-14621, -12

In [45]:
print(found, len(found))

[[-15175, -10832], [-11310, -14354], [-14620, -12594], [-14621, -12589], [-13843, -13476], [-13844, -13473], [-15032, -11712], [-15033, -11707], [-13213, -13915], [-13216, -13913], [-14281, -13034], [-14282, -13031], [-14859, -12151], [-14863, -12148], [-15140, -11275], [-15141, -11254], [-12719, -14135], [-12725, -14134], [-13574, -13696], [-13577, -13693], [-14075, -13255], [-14078, -13252], [-14466, -12813], [-14467, -12809], [-14752, -12370], [-14753, -12365], [-14954, -11933], [-14955, -11929], [-15093, -11493], [-15095, -11486], [-15164, -11068], [-15167, -11036], [-12350, -14245], [-12354, -14244], [-12988, -14025], [-12990, -14023], [-13399, -13805], [-13400, -13804], [-13717, -13586], [-13720, -13581], [-13965, -13364], [-13969, -13361], [-14181, -13145], [-14182, -13142], [-14375, -12924], [-14378, -12921], [-14545, -12703], [-14551, -12698], [-14689, -12480], [-14691, -12474], [-14809, -12262], [-14810, -12251], [-14910, -12042], [-14912, -12035], [-14998, -11822], [-14999, 

In [46]:
def insertinto(a, line):
    from math import floor as fl
    import numpy as np
    #first find middle
    if a is None:
        return np.array(line).reshape(1,-1)
    start = 0
    end = a.shape[0]
    
    if line[0] > a[-1,0]:
        return np.vstack((a,line))
    if line[0] < a[0,0]:
        return np.vstack((line,a))
    while end > start:
        pull = fl((end+start)/2.0)
        if a[pull,0] == line[0]:
            break
        elif a[pull,0] < line[0]:
            if a[pull+1,0] > line[0]:
                return np.insert(a,pull+1,line,axis=0)
            start = pull+1
        else:
            if a[pull-1,0] < line[0]:
                return np.insert(a,pull,line,axis=0)
            end = pull-1
    #now go backwards:
    anchor = pull
    if end == start:
        anchor = end
        pull = anchor
    while True:
        pull -= 1
        if pull == -1:
            anchorstart = 0
            break
        if a[pull,0] != line[0]:
            anchorstart = pull + 1
            break
    pull = anchor
    while True:
        pull += 1
        if pull == a.shape[0]:
            anchorend = pull - 1
            break
        if a[pull,0] != line[0]:
            anchorend = pull - 1
            break
    start = anchorstart
    end = anchorend
    
    level1start = anchorstart
    level1end = anchorend
    
    
    if line[1] > a[end,1]:
        return np.insert(a,end+1,line,axis=0)
    if line[1] < a[start,1]:
        return np.insert(a,start,line,axis=0)
    while True:
        pull = fl((end+start)/2.0)
        if a[pull,1] == line[1]:
            break
        elif a[pull,1] < line[1]:
            if a[pull+1,1] > line[1]:
                return np.insert(a,pull+1,line,axis=0)
            start = pull+1
        else:
            if a[pull-1,1] < line[1]:
                return np.insert(a,pull,line,axis=0)
            end = pull-1
    anchor = pull
    if end == start:
        anchor = end
        pull = anchor
    print(anchor)
    while True:
        pull -= 1
        if pull == -1:
            anchorstart = level1start
            break
        if a[pull,1] != line[1]:
            anchorstart = pull + 1
            break
    pull = anchor
    
    while True:
        pull += 1
        if pull == a.shape[0]:
            anchorend = level1end
            break
        if a[pull,1] != line[1]:
            anchorend = pull - 1
            break
    start = anchorstart
    end = anchorend
    

    if line[2] > a[end,2]:
        return np.insert(a,end+1,line,axis=0)
    if line[2] < a[start,2]:
        return np.insert(a,start,line,axis=0)
    while True:
        pull = fl((end+start)/2.0)
        if a[pull,2] == line[2]:
            print("FUCKING CRY")
        elif a[pull,2] < line[2]:
            if a[pull+1,2] > line[2]:
                return np.insert(a,pull+1,line,axis=0)
            start = pull+1
        else:
            if a[pull-1,2] < line[2]:
                return np.insert(a,pull,line,axis=0)
            end = pull-1

In [47]:
r = None
for i in found:
    r = insertinto(r,i)
print(r)

[[-15175 -10832]
 [-15173 -10879]
 [-15172 -10901]
 ...
 [-11485 -14350]
 [-11435 -14353]
 [-11310 -14354]]
